# Conducting a Quantum Speed Limit Search using Cluster Nodes for the Optimization Protocol.

<font size="5"> 
This notebook employs a methodology for determining speed limits that is analogous to the approach utilized by the "QSLsearch_EntanglingGates" notebook. Nevertheless, this example demonstrates how the use of multiple cluster nodes can facilitate the completion of the task in question, as opposed to the parallelisation of operations on a single computer. The successful execution of the optimisation protocol necessitates the execution of a batch script subsequent to the creation of all requisite run folders (as detailed in this notebook).

In [ ]:
import os

import copy
"""
Assignment statements in Python do not copy objects!
they create bindings between a target and an object.
a copy is sometimes needed so one can change one copy without changing the other!
"""
from pathlib import Path

import numpy as np
import scipy
import qutip
from qutip import tensor,qeye,ket
from qutip.qip.operations import berkeley,swap

import krotov


import matplotlib.pyplot as plt

import qdyn.model
import qdyn.pulse

import subprocess as subp

%matplotlib widget

In [3]:
#final time
τ = 1.8


#t_rise
tr = τ/10

#delta_t
Δt = 0.0018

#lambda
λ = 0.07

# Number of time points
n = int(τ/Δt + 1)


In [4]:
#Target-Gate

B = berkeley(2)

E = qeye(2)

S = swap(2)

U = tensor(S,E)*tensor(E,B)*tensor(S,E)


Target = U

Target = np.array(Target)

target_gate = qutip.Qobj(Target)

# Initialize List
L = [τ,tr,Δt,λ,target_gate]


In [5]:
# Here, the Hamiltonian of the asymmetric chain imposed with the two local controls is numerically implemented.

def get_Ham(ν1=5.0, ν2 = 5.3, ν3 = 5.5, T = τ, t_rise = tr): 
    
    # N , normalization constant for the krotov.shapes.flattop function
    N = 1.131234587427
    u1 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν1*t*2*np.pi))
    u2 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν2*t*2*np.pi)) 
    u3 = lambda t, args: N*krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*4*np.pi/T*(np.cos(ν3*t*2*np.pi))
                                                                                                                                                                                                                 
    # Pauli matrices
    Z = qutip.operators.sigmaz()
    X = qutip.operators.sigmax()
    E = qeye(2) 

   # qubit-qubit interactions and their interaction strengths e.g. X1X2 and k1
    X1X2 = tensor(X,X,E)
    Z1Z2 = tensor(Z,Z,E)
    X2X3 = tensor(E,X,X)
 
    k1 = 0.11
    k2 = 0.22
    k3 = 0.31

    """
    While drift describes the three uncoupled qubits, 
    coupl describes the qubit-qubit interactions in the asymmetric chain
    The total time independent Hamiltonian H0, is the sum of both terms:
    """
    
    
    drift = -1/2 * ( ν1*tensor(Z,E,E) + ν2*tensor(E,Z,E) + ν3*tensor(E,E,Z) ) 
    coupl = k1*X1X2 + k2*Z1Z2 + k3*X2X3 
    
    H0 = drift + coupl
    H0 = np.array(H0)  
    
    Ham_0 = qutip.Qobj(H0)

    # Numerically, it is important to transform H0 into a numpy array and subsequently to a qutip. quantum object,
    # such that the optimizer recognizes Ham_0 as a 16x16 matrix.

    """
    Define local control operators seperately.
    The output is a nested list, which will be passed to the optimizer.
    """

    H1 = tensor(X,E,E)
    H1 = np.array(H1)
    Ham_1 = qutip.Qobj(H1)
    

    H2 = tensor(E,X,E)
    H2 = np.array(H2)
    Ham_2 = qutip.Qobj(H2)
    
    H3 = tensor(E,E,X)
    H3 = np.array(H3)
    Ham_3 = qutip.Qobj(H3)

    
    return [Ham_0, [Ham_1, np.vectorize(u1)],[Ham_2, np.vectorize(u2)],[Ham_3, np.vectorize(u3)]]

In [6]:
# Define Model

def qdyn_model(
    Ham,
    T, # final_time
    nt, # number of time steps
    basis, # logical basis
    gate, # target gate
    runfolder,
    prop_method="exact",
    J_T_conv=1e-4,
    Δ_J_T_conv = 1e-7,
    iter_stop=100000,
    **pulse_oct_kwargs, # kwargs = keyword arguments
):
    """ Create qdyn-model

    """
    runfolder = Path(runfolder) # path to runfolder
    ! mkdir $runfolder # bash command 
    tgrid = qdyn.pulse.pulse_tgrid(T=T, nt=nt, t0=0)
      # For technical reasons, QDYN stores the pulses on the intervals of the time grid.
    # That means, the time grid for the pulse needs to be shifted
    #by dt/2 with respect to the time grid used for the propagation
    
    model = qdyn.model.LevelModel()
    # 
    for H in Ham:
        if isinstance(H, list):
            model.add_ham(
                H[0],
                pulse=qdyn.pulse.Pulse(
                    tgrid,
                    amplitude=H[1](tgrid, None),
                    time_unit="ns",
                    ampl_unit="GHz",
                    config_attribs={
                        **pulse_oct_kwargs
                    }, # Additional config data, for when generating a QDYN config
                    # file section describing the pulse (e.g. {'oct_shape': 'flattop', 't_rise': '10_ns'})
                ),
                op_unit="iu",
            )
        else:
            model.add_ham(H, op_unit="GHz")
            
    qdyn.io.write_cmplx_array(
        qutip.Qobj(gate).full().flatten(order="F"), str(runfolder/"gate.dat")
    )
    model.set_propagation(
        T, nt, time_unit="ns", prop_method=prop_method
    )
    model.set_oct("krotovpk", J_T_conv, 5000, iter_stop=iter_stop, continue_=False, delta_J_T_conv=Δ_J_T_conv)
    
    # Observables
    Z = qutip.operators.sigmaz()
    E = qeye(2)
    
    Z1 = qutip.Qobj(np.array(tensor(Z,E,E)))
    Z2 = qutip.Qobj(np.array(tensor(E,Z,E)))
    Z3 = qutip.Qobj(np.array(tensor(E,E,Z)))
    



    model.add_observable(Z1, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "1" )
    model.add_observable(Z2, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "2" )
    model.add_observable(Z3, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "3" )
    
    
    
    
    for i, b in enumerate(basis):
        model.add_state(b, f"basis{i}")
        

    Path(runfolder).mkdir(exist_ok=True, parents=True)
    model.write_to_runfolder(str(runfolder))

In [ ]:
# "create_runfolder":
# creates a runfolder that contains information about the system, its dynamics, numerical parameters for the upcoming Krotov optimization with final time τ

# "do_final_times":
# For the final time a = τ , we reduce c times the parameter b from a. The output is an array that consists of final times: a, a-b, a-2b, ... a-(c-1)b.
# These are c final times in total, which will be passed to the subsequent function "do_optimization_input"

# "do_optimization_input":
# Here, for each of the c final times created in # "do_final_times" lists are created. Every list P consist of a final_time, t_rise, delta_t, lambda_0 and the target gate.
# In other words, the list P contains information necessary for performing a single optimization at certain final time.

# "make_multiple_runfolders": 
# The aim is to run each of the c Krotov optimization on a different cluster node. Ideally, all krotov optimizations will be performed simultaneously.
# Therefore, this function creates c runfolders (all runfolders correspond to different final times T, T-b, T-2b,...) that will be called for optimization, when the batch script runs.



def create_runfolder(L):
    
    [τ,tr,Δt,λ,target_gate] = L

    pulse_oct_kwargs = dict(oct_lambda_a=λ, oct_shape = "flattop",t_rise=qdyn.units.UnitFloat(τ/10, "ns"), t_fall=qdyn.units.UnitFloat(τ/10, "ns") )
    
    rf_p = "rf"+str(τ)
    
    qdyn_model(get_Ham(T = τ, t_rise = τ/10), τ,  int(τ/Δt + 1),[qutip.basis(8,i) for i in range(0,8)], target_gate, rf_p  , **pulse_oct_kwargs)

   

def do_final_times(a, b, c):
    """
    a = final time
    b = decreasing parameter
    c = how often shall b be reduced from a
    """
    return np.arange(a, 0, -b)[:c]

def do_optimization_input(L,b,c):
 
    a = L[0]
    P = [[m] + L[1:5] for m in do_final_times(a, b, c)]
    
    return P

def make_multiple_runfolders(L,b,c):
    input_list = do_optimization_input(L,b,c)
    
    for i,L_i in enumerate(input_list):
        rf_i = create_runfolder(L_i)
    print('Ready')

#make_multiple_runfolders(L,0.2,6)


#! mkdir All_runfolders
#! mkdir job_msgs
    